<a href="https://colab.research.google.com/github/KickDirtBaseball/register/blob/master/Updated_Framing_Leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pybaseball --upgrade
from pybaseball import statcast
import pandas as pd
from joblib import load

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 42.9 MB/s eta 0:00:00


In [3]:
# 2025 April example — adjust as needed
df = statcast(start_dt="2025-04-01", end_dt="2025-04-30")

This is a large query, it may take a moment to complete


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = d

In [4]:
df = df[df['description'].isin(['called_strike', 'ball'])].copy()
df = df[df['player_name'].notna()]
df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)
df['hand_matchup'] = df['p_throws'] + '_' + df['stand']

In [6]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/framing_model_2025 2V.04.joblib"
preprocessor_path = "/content/drive/MyDrive/FramingPreprocessorV.4.joblib"

framing_model = load(model_path)
preprocessor = load(preprocessor_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
features = [
    'plate_x', 'plate_z', 'sz_top', 'sz_bot', 'zone',
    'release_extension', 'release_pos_x', 'release_pos_z', 'release_speed',
    'pitch_type', 'hand_matchup', 'count', 'stand', 'p_throws'
]

df = df.dropna(subset=features)
X = preprocessor.transform(df[features])
df['strike_prob'] = framing_model.predict_proba(X)[:, 1]
df['is_strike'] = (df['description'] == 'called_strike').astype(int)
df['framing_value'] = df['is_strike'] - df['strike_prob']

In [8]:
leaderboard = (
    df.groupby('player_name')
    .agg(
        total_pitches=('framing_value', 'count'),
        total_value=('framing_value', 'sum'),
        value_per_100=('framing_value', lambda x: 100 * x.sum() / len(x))
    )
    .sort_values('value_per_100', ascending=False)
    .reset_index()
)

leaderboard.head(15)

,player_name,total_pitches,total_value,value_per_100
0,"Hedges, Austin",19,3.30,17.368421
1,"Koss, Christian",2,0.30,15.000000
2,"Guerrero, Luis",13,1.02,7.846154
3,"Schultz, Paxton",58,4.55,7.844828
4,"Wilson, Will",15,1.16,7.733333
5,"Hill, Jaden",29,1.79,6.172414
6,"Alexander, Jason",27,1.65,6.111111
7,"Wiles, Nathan",8,0.46,5.750000
8,"Leiter Jr., Mark",92,4.92,5.347826
9,"Bowlan, Jonathan",11,0.54,4.909091
